# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Pull the CSV file from the WeatherPy Notebook
vaca = "../output_data/cities_mike.csv"
vaca_df = pd.read_csv(vaca, encoding="ISO-8859-1")
vaca_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Maniago,46.17,12.71,75.99,69,18,0.83,IT,1595186617
1,Yangi Marg`ilon,40.43,71.72,78.80,36,0,6.93,UZ,1595186617
2,WaddÄn,29.16,16.14,85.28,43,0,23.11,LY,1595186617
3,Ribeira Grande,38.52,-28.70,73.40,64,40,9.17,PT,1595186617
4,Tasiilaq,65.61,-37.64,53.60,58,0,9.17,GL,1595186617
...,...,...,...,...,...,...,...,...,...
536,Iwanai,42.97,140.51,64.15,90,97,2.26,JP,1595186667
537,Gushikawa,26.35,127.87,82.40,94,1,4.70,JP,1595186667
538,Culebra,18.30,-65.30,89.60,59,20,13.87,PR,1595186667
539,Srostki,52.42,85.70,58.42,84,99,2.64,RU,1595186667


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vaca_df[["Lat", "Lng"]]
humid = vaca_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Filter the list for ideal conditions
ideal_vaca = vaca_df[(vaca_df["Max Temp"]<=85) &
                     (vaca_df["Max Temp"]>=75) &
                     (vaca_df["Wind Speed"]<=10) &
                     (vaca_df["Cloudiness"]==0)]
ideal_vaca

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Yangi Marg`ilon,40.43,71.72,78.80,36,0,6.93,UZ,1595186617
108,Nurota,40.56,65.69,82.40,18,0,2.24,UZ,1595186626
130,Baculin,7.45,126.58,78.94,78,0,3.94,PH,1595186627
155,Aleksandriyskaya,44.23,43.35,80.60,42,0,2.24,RU,1595186629
182,Heraclea,37.51,34.05,80.71,40,0,3.18,TR,1595186632
195,Poso,-1.40,120.75,76.98,75,0,5.17,ID,1595186633
201,Talas,38.69,35.55,80.60,34,0,1.12,TR,1595186633
219,Kavos,38.83,22.83,79.00,66,0,1.57,GR,1595186635
246,Llanes,43.42,-4.75,75.00,66,0,1.88,ES,1595186637
278,Pirgos,35.12,25.72,78.01,51,0,3.00,GR,1595186642


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create the dataframe of Hotels
hotel_df = ideal_vaca[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Yangi Marg`ilon,UZ,40.43,71.72,
108,Nurota,UZ,40.56,65.69,
130,Baculin,PH,7.45,126.58,
155,Aleksandriyskaya,RU,44.23,43.35,
182,Heraclea,TR,37.51,34.05,
195,Poso,ID,-1.40,120.75,
201,Talas,TR,38.69,35.55,
219,Kavos,GR,38.83,22.83,
246,Llanes,ES,43.42,-4.75,
278,Pirgos,GR,35.12,25.72,


In [7]:
#locations_ideal = ideal_vaca[["Lat", "Lon"]]
target_search = "lodging"
#coords = locations_ideal
radius = 5000

params = {
    #"location": coords,
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# Use the lat/lng in ideal_vaca to identify hotels
for index, row in ideal_vaca.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)
    response = response.json()
    #We need to get the first hotel from the response and put it in Hotel Name
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Get the next hotel...")
    

#print(response)
hotel_df

Get the next hotel...
Get the next hotel...
Get the next hotel...
Get the next hotel...
Get the next hotel...


,City,Country,Lat,Lng,Hotel Name
1,Yangi Marg`ilon,UZ,40.43,71.72,Hotel Mehmon Saroy
108,Nurota,UZ,40.56,65.69,Guest house Nur Eco chashma turizm
130,Baculin,PH,7.45,126.58,
155,Aleksandriyskaya,RU,44.23,43.35,
182,Heraclea,TR,37.51,34.05,Mirel Hotel
195,Poso,ID,-1.40,120.75,Ancyra Hotel By Continent - Poso
201,Talas,TR,38.69,35.55,Radisson Blu Hotel Kayseri
219,Kavos,GR,38.83,22.83,
246,Llanes,ES,43.42,-4.75,Don Paco
278,Pirgos,GR,35.12,25.72,Mistral Mare Hotel


In [8]:
pprint(response)

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAF92iN8HjYtoBw7be-Zjvt_0v1WyyGXeclB39487hHKY9jVEIVrbt4NX0WjZ3yBlpCkvkALAN4XKtI_or4Qm8kpCVG5uuBjqdLtaF5vIp3rZfliXSOm-hOFT1ZYPpB48ep-cRqiju2eJplbZ7Gdw92YIQyltn60yBYDSf_AIPe0EnQkVivifjo2-IOnzm12WMqpANud27HzOc9b6foKH6W4hRc9pfxv6moprFCP1fv4qNsdi9sRbEAIfasVX9F6P5EoVmpfv0s9uzOYFE2DZnKQxt924e9CBZF1-QJhfHo7W9yAt-RM0V1tpouxQh6wQnBXo_rQZZC4c7SL3pw12IncJvpw7JU8Zz1-yVJyC9hkbbTOEDkFQT2LkQfWJcCl5RxIQWY-Esaxaup4yB9GnZl0IXBoUQ7-xJtGvLyGk8eOYBWxOUAlhORM',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 43.6174453, 'lng': 51.213423},
                           'viewport': {'northeast': {'lat': 43.61937602989272,
                                                      'lng': 51.21532697989272},
                                        'southwest': {'lat': 43.61667637010728,
                                                      'lng': 51.21262732010727}}},
              'icon': 'https://maps.gstatic.com/mapfi

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(layer)

# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))